# NTIRE 2024 - Portrait Quality Assessment Challenge ft. DXOMARK


# -- INSTRUCTIONS --
# **[READ CAREFULLY]**
 

# 1 - DESCRIPTION
## What is this?
This is a simple instruction notebook to help you submit your model.

## Links
- **NTIRE 24 challenge**: https://codalab.lisn.upsaclay.fr/competitions/17311#participate
- **PIQ23 github**: https://github.com/DXOMARK-Research/PIQ2023
- **PIQ23 Download link**: https://corp.dxomark.com/data-base-piq23/

## Test set
- The evaluation process consists of testing your model on an internat portrait dataset of around ***200 scenes of 7 images each, each shot with a different device of close quality***.
- Images are either jpeg or TIFF with extensions ***'.jpg' or '.tiff'***.
- Images are either landscape or portrait with a size of: ***1280x960*** or ***960x1280***.
- Lighting conditions are: ***Indoor, Outdoor, Lowlight and Night***.
- Different skintones, genders and ages are used. Prepare for something slightly different than PIQ23.
- Do not excpect the same people to be present in the internal dataset.
- The test csv will include ***image names and the categories of each class alongside the lighting conditions***. *Please refer to the images.csv*

## Hardware requirements
- You are free to do inference on one or multiple images.
- Please make sure that your model is able to run on a ***single 8GB VRAM GPU***.
- Maximum Teraflops: ***5TFLOPS*** *(Refer below to calculate teraflops on your model)*.
- Maximum inference time: ***5 seconds/image***. *Model Loading does not count*.
- Maximum model.pth size: ***2GB***.
- Maximum RAM: **16GB**.
- **NOTE: If your model comply with the 16GB RAM and 5s/image on cpu, you don't need to use GPU**

## Submission
- You need to submit a script with the following naming: ***model_[LAST_NAME]_[FIRST_NAME].py***
- Your script will be put in the ***./models/ folder***.
- Your model weights should be saved in ***./weights/weights_[LAST_NAME]_[FIRST_NAME].(pth,ckpt)***
- Your assets will be in ***./assets/assets_[LAST_NAME]_[FIRST_NAME]***.
- You will get access to one level above the models/ folder.
- Your script needs to load the ***./images.csv***.
- Images paths are structured as follows: ***images/[class]/[imagename.(jpg,tiff)]***.
- You need to save your results as follows: ***./results/result_[LAST_NAME]_[FIRST_NAME].csv***.
- You need to add a ***column 'SCORE' to the images.csv***. *KEEP ALL OTHER METADATA*.
- You can use a ***comma or semi-colon separator for the results***. Any other separator will not be considered.
- Refer to *./models/model_23_PIQ.py* for an example.

# 2 - ASSETS

## CONSTANTS

In [2]:
EXTENSIONS = ('.tiff', '.tif', '.TIFF', '.TIF', '.jpg', '.JPG', '.jpeg')
CONDITIONS = ('OUTDOOR', 'INDOOR', 'LOWLIGHT', 'NIGHT')
IMAGE_SIZE = ((1280, 960), (960, 1280))

## Hardware Check

In [1]:
import torch
from torchvision.models import resnet50, ResNet50_Weights
from thop import profile # pip install thop

def torch_cuda_memory_usage():
    """Returns CUDA memory usage if available"""
    if torch.cuda.is_available():
        torch.cuda.synchronize()  # Wait for all CUDA kernels to finish
        allocated_memory = torch.cuda.memory_allocated()  # Total allocated memory
        cached_memory = torch.cuda.memory_reserved()  # Total cached memory
        return allocated_memory / (1024**3), cached_memory / (1024**3)  # Convert bytes to GB
    else:
        return 0, 0

def test_model_resources(model, batch):
    
    macs, params = profile(model, inputs=(batch, ), verbose=False)
    flops = macs * 2  # Convert MACs to FLOPs
    tflops = flops / (10**12)  # Convert FLOPs to TFLOPs  
    
    torch.cuda.reset_peak_memory_stats()  # Reset peak memory stats for accurate peak measurement

    # Measure memory before inference
    allocated_before, cached_before = torch_cuda_memory_usage()
    
    model = model.cuda()  # Move model to GPU
    batch = batch.cuda()  # Move data to GPU
    
    # Dummy forward pass to measure VRAM usage
    with torch.no_grad():
        _ = model(batch)
        
    # Measure memory after inference
    allocated_after, cached_after = torch_cuda_memory_usage()
    peak_allocated = torch.cuda.max_memory_allocated() / (1024**3)  # Peak allocated memory during inference
    
    vram_usage_allocated = allocated_after - allocated_before  # Approximation of additional VRAM used during inference
    vram_usage_cached = cached_after - cached_before  # Approximation based on cached memory

    print(f"MACs: {macs}")
    print(f"FLOPs: {flops}")
    print(f"TFLOPs: {tflops}")
    print(f"Approx. Additional VRAM Usage (Allocated) during Inference: {vram_usage_allocated} GB")
    print(f"Approx. Additional VRAM Usage (Cached) during Inference: {vram_usage_cached} GB")
    print(f"Peak VRAM Usage during Inference: {peak_allocated} GB")
    
    del model, batch  # Free up memory
    torch.cuda.empty_cache()  # Clear cache

In [2]:

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1) # change to your model
batch_size = 18 # Test the batch size you want
batch = torch.stack([torch.randn(3, 1280, 960)]*batch_size)

test_model_resources(model, batch)

MACs: 1821354430464.0
FLOPs: 3642708860928.0
TFLOPs: 3.642708860928
Approx. Additional VRAM Usage (Allocated) during Inference: 0.3515634536743164 GB
Approx. Additional VRAM Usage (Cached) during Inference: 5.64453125 GB
Peak VRAM Usage during Inference: 5.28789758682251 GB


# 3 - SUBMISSION

In [1]:
%run ./models/model_23_PIQ.py

Hello, I am an asset.
